In [ ]:
using DifferentialEquations
using PyPlot
using ReactionMechanismSimulator

In [ ]:
phaseDict = readinput("../src/testing/superminimal.rms") #load mechanism dictionary

In [ ]:
spcs = phaseDict["gas"]["Species"]; #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
rxns = phaseDict["gas"]["Reactions"];

In [ ]:
ig = IdealGas(spcs,rxns,name="gas"); #Define the phase (how species thermodynamic and kinetic properties calculated)
initialconds = Dict(["T"=>1000.0,"ts"=>[0.0,0.02,0.04,0.05],"V"=>[0.008314,0.01,0.012,0.012],"H2"=>0.67,"O2"=>0.33]); #Set simulation Initial Temp and Pressure
#f(x::Float64) = 0.008314
#initialconds = Dict(["T"=>1000.0,"V"=>f,"H2"=>0.67,"O2"=>0.33]); #Set simulation Initial Temp and Pressure
domain,y0 = ParametrizedVDomain(phase=ig,initialconds=initialconds); #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
react = Reactor(domain,y0,(0.0,0.0501)); #Create the reactor object

In [ ]:
sol = solve(react.ode,DifferentialEquations.CVODE_BDF(),abstol=1e-20,reltol=1e-8); #solve the ode associated with the reactor

In [ ]:
bsol = Simulation(sol,domain);

In [ ]:
plotmolefractions(bsol,0.05; t0=1e-15, N=1000, tol=0.01)

In [ ]:
ts = exp.(range(log(1e-15),length=1000,stop=log(0.05)))
plot(ts,[sol(t)[domain.indexes[end]] for t in ts]) #Temperature vs time

In [ ]:
ts[argmax(diff([sol(t)[end] for t in ts]))] #Ignition Delay Time in seconds

In [ ]:
getfluxdiagram(bsol,0.036,centralspecieslist=["OH(D)"],radius=5)

In [ ]:
plotrops(bsol,"OH(D)",0.01;tol=0.001)